# Douyin Live Recorder

Notebook นี้ใช้สำหรับรัน DouyinLiveRecorder เพื่อบันทึกวิดีโอไลฟ์จาก Douyin (TikTok จีน)

In [ ]:
# Clone repositories
!git clone https://github.com/ihmily/DouyinLiveRecorder.git
!git clone https://github.com/chaiz64/l777k.git

In [ ]:
# Copy configuration files
!cp l777k/config.ini DouyinLiveRecorder/config/
!cp l777k/URL_config.ini DouyinLiveRecorder/config/

In [ ]:
# เข้าไปยังโฟลเดอร์โปรเจค
%cd DouyinLiveRecorder

# ติดตั้ง dependencies
!pip3 install -r requirements.txt

In [ ]:
# ติดตั้ง ffmpeg
!apt update
!apt install ffmpeg -y

In [ ]:
# รันโปรแกรม
!python main.py

In [ ]:
import time
import datetime

def keep_colab_alive(duration_hours=24):
    """
    Keeps the Google Colab runtime active by running a loop.

    Args:
        duration_hours (int): The duration in hours for which the script
                              should attempt to keep the runtime alive.
                              Default is 24 hours. Set to 0 for indefinite run
                              (requires manual stop).
    """
    print(f"Script will attempt to keep Colab alive for {duration_hours} hours. (Set to 0 for indefinite run)")
    start_time = datetime.datetime.now()
    end_time = start_time + datetime.timedelta(hours=duration_hours)

    counter = 0
    try:
        while True:
            current_time = datetime.datetime.now()
            if duration_hours > 0 and current_time >= end_time:
                print(f"Maximum duration of {duration_hours} hours reached. Stopping script.")
                break

            counter += 1
            print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] Colab is alive! Iteration: {counter}")
            # Sleep for a short period to prevent excessive CPU usage
            time.sleep(60) # Sleep for 60 seconds (1 minute)

    except KeyboardInterrupt:
        print("\nScript stopped manually by user (Ctrl+C).")
    except Exception as e:
        print(f"\nAn error occurred: {e}")
    finally:
        print("Colab keep-alive script finished.")

# --- How to use ---
# Call the function to start keeping Colab alive.
# You can specify the duration in hours.
# For example, to keep it alive for 12 hours:
# keep_colab_alive(duration_hours=12)

# To run indefinitely until manually stopped (Ctrl+C):
keep_colab_alive(duration_hours=0) # Set to 0 for indefinite run


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import google.colab.files
import time

# --- Setup and Initialization ---
# Path to your download folder
# Initial default path, user can change this via UI
default_download_dir = "/content/DouyinLiveRecorder/downloads/抖音直播/"
download_limit_per_batch = 1 # Limit downloads to 3 files at a time

# List of common video file extensions to support
video_extensions = ('.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm', '.ogg', '.3gp', '.ts')

# Create a temporary download folder (if necessary)
os.makedirs("download_temp", exist_ok=True)

# File name for saving downloaded URLs
download_links_file = "downloaded_links.txt"

# Ensure the download links file exists
if not os.path.exists(download_links_file):
    with open(download_links_file, "w") as f:
        f.write("") # Create an empty file

# --- Helper Functions ---
def get_video_info(file_path):
    """
    Function to retrieve video information (size and duration).
    In a real scenario, you might use libraries like moviepy or ffprobe
    to extract this information from actual video files.
    For this example, we will simulate the data.
    """
    file_size = 0
    duration_seconds = 0
    file_name = os.path.basename(file_path)

    try:
        file_size = os.path.getsize(file_path) # File size in bytes
        # Simulate duration from file name, or use random/default values
        duration_match_str = next((s for s in file_name.split('_') if 's' in s and s.replace('s', '').isdigit()), None)
        if duration_match_str:
            duration_seconds = int(duration_match_str.replace('s', ''))
        else:
            duration_seconds = 600 + (os.path.getsize(file_path) % 600) # At least 10 minutes (600 seconds)
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"⚠️ Error getting info for {file_path}: {e}")
        return None

    return {
        "name": file_name,
        "path": file_path,
        "size_mb": round(file_size / (1024 * 1024), 2),
        "duration_min": round(duration_seconds / 60, 2),
        "raw_duration_sec": duration_seconds
    }

def create_dummy_files(base_dir, num_files=10):
    """Creates dummy files for testing if the actual folder is empty"""
    print(f"✨ Creating dummy files in {base_dir} for testing...")
    os.makedirs(base_dir, exist_ok=True)
    for i in range(1, num_files + 1):
        duration = 5 if i % 3 == 0 else (600 + (i * 30)) # Some files have duration less than 10 minutes
        ext = video_extensions[i % len(video_extensions)]
        size = 1024 * 1024 * (i * 5) # Dummy file size (5MB, 10MB, ...)
        file_name = f"video_{i}_duration_{duration}s{ext}"
        file_path = os.path.join(base_dir, file_name)
        with open(file_path, "wb") as f:
            f.write(os.urandom(size)) # Write random data of specified size
    print(f"✅ Created {num_files} dummy files successfully.")

# --- Main Application Class ---
class ColabVideoDownloader:
    def __init__(self, download_dir, download_limit_per_batch, video_extensions, download_links_file):
        self.download_dir = download_dir
        self.download_limit_per_batch = download_limit_per_batch
        self.video_extensions = video_extensions
        self.download_links_file = download_links_file
        self.video_files_info = []
        self.skip_checkboxes = []
        self.current_filter_text = ""

        # UI Elements
        self.output_area = widgets.Output()
        self.dir_input = widgets.Text(
            value=self.download_dir,
            placeholder='Enter your download folder path',
            description='Folder:',
            layout=widgets.Layout(width='auto')
        )
        self.refresh_button = widgets.Button(
            description="🔄 Refresh Files",
            button_style='primary',
            tooltip="Scan the download folder again"
        )
        self.clear_links_button = widgets.Button(
            description="🗑️ Clear Download History",
            button_style='danger',
            tooltip="Clear the downloaded_links.txt file"
        )
        self.filter_input = widgets.Text(
            value="",
            placeholder='Search file name...',
            description='Search:',
            layout=widgets.Layout(width='auto')
        )
        self.single_download_container = widgets.VBox([])
        self.batch_download_container = widgets.VBox([])
        self.download_history_output = widgets.Output()

        self._setup_ui()
        self._setup_event_handlers()
        self._scan_files()
        self._update_ui_display()
        self._display_download_history()

    def _setup_ui(self):
        """Sets up the main UI layout."""
        # FIX: Use widgets.HTML instead of IPython.display.HTML for elements directly in VBox
        header = widgets.HTML("<h1>🚀 Colab Video Downloader (Epic Version)</h1>")
        instructions = widgets.HTML("""
            <p>Welcome to the advanced video downloader tool! You can:</p>
            <ul>
                <li><b>Specify Download Folder:</b> Enter the path to your video folder.</li>
                <li><b>Refresh Files:</b> Scan for new video files.</li>
                <li><b>Download Single File:</b> Click the download button for each video.</li>
                <li><b>Batch Download:</b> Select files to skip and download the rest.</li>
                <li><b>Search:</b> Filter the video list by name.</li>
                <li><b>View Download History:</b> Check which files have already been downloaded.</li>
            </ul>
            <p><b>Note:</b> Video files shorter than 10 minutes (600 seconds) will be skipped.</p>
        """)

        dir_control_box = widgets.HBox([self.dir_input, self.refresh_button])
        filter_box = widgets.HBox([self.filter_input, self.clear_links_button])

        self.main_layout = widgets.VBox([
            header,
            instructions,
            dir_control_box,
            filter_box,
            widgets.HTML("<h2>--- ⬇️ Download Single File ---</h2>"), # FIX: Also ensure these are widgets.HTML
            self.single_download_container,
            widgets.HTML("<h2>--- 📦 Batch Download ---</h2>"), # FIX: Also ensure these are widgets.HTML
            self.batch_download_container,
            widgets.HTML("<h2>--- 📜 Download History ---</h2>"), # FIX: Also ensure these are widgets.HTML
            self.download_history_output,
            self.output_area # For general messages and download feedback
        ])

    def _setup_event_handlers(self):
        """Sets up event handlers for UI widgets."""
        self.refresh_button.on_click(self._on_refresh_button_click)
        self.clear_links_button.on_click(self._on_clear_links_button_click)
        self.filter_input.observe(self._on_filter_input_change, names='value')
        self.dir_input.observe(self._on_dir_input_change, names='value')

    def _on_dir_input_change(self, change):
        """Updates the download directory when the text input changes."""
        self.download_dir = change['new']
        with self.output_area:
            clear_output(wait=True)
            print(f"Download directory set to: {self.download_dir}")
        self._scan_files()
        self._update_ui_display()

    def _on_refresh_button_click(self, b):
        """Handles the refresh button click."""
        with self.output_area:
            clear_output(wait=True)
            print("Refreshing file list...")
        self._scan_files()
        self._update_ui_display()
        print("✅ Refresh complete!")

    def _on_clear_links_button_click(self, b):
        """Handles clearing the downloaded links file."""
        with self.output_area:
            clear_output(wait=True)
            try:
                with open(self.download_links_file, "w") as f:
                    f.write("")
                print("✅ Download history cleared!")
                self._display_download_history()
            except Exception as e:
                print(f"❌ Error clearing history: {e}")

    def _on_filter_input_change(self, change):
        """Filters the displayed files based on user input."""
        self.current_filter_text = change['new'].lower()
        self._update_ui_display()

    def _scan_files(self):
        """Scans the specified directory for video files."""
        self.video_files_info = []
        if not os.path.exists(self.download_dir) or not os.listdir(self.download_dir):
            with self.output_area:
                print(f"⚠️ Folder {self.download_dir} not found or is empty. Creating dummy files...")
            create_dummy_files(self.download_dir) # Create dummy files if no actual files

        for root, _, files in os.walk(self.download_dir):
            for file in files:
                if file.lower().endswith(self.video_extensions):
                    file_path = os.path.join(root, file)
                    info = get_video_info(file_path)
                    if info and info["raw_duration_sec"] >= 600: # Skip files with duration less than 10 minutes
                        self.video_files_info.append(info)

        self.video_files_info.sort(key=lambda x: x['name'])
        with self.output_area:
            print(f"\n🎬 Found {len(self.video_files_info)} videos available for download (length >= 10 minutes):")
            if not self.video_files_info:
                print("   ❌ No video files matching the criteria (length >= 10 minutes) in the specified folder.")
            else:
                for video in self.video_files_info:
                    print(f"   - {video['name']} | Size: {video['size_mb']} MB | Duration: {video['duration_min']} minutes")

    def _update_ui_display(self):
        """Updates the displayed file lists based on current filters."""
        filtered_files = [
            file_info for file_info in self.video_files_info
            if self.current_filter_text in file_info['name'].lower()
        ]

        # Update Single File Download section
        single_widgets = []
        if not filtered_files:
            single_widgets.append(widgets.HTML("<p>🚫 No video files available for single download.</p>"))
        else:
            for video in filtered_files:
                single_widgets.append(self._create_single_download_widget(video))
        self.single_download_container.children = single_widgets

        # Update Batch Download section
        self.skip_checkboxes = [] # Reset checkboxes for filtered list
        if not filtered_files:
            self.batch_download_container.children = [widgets.HTML("<p>🚫 No video files available for batch download.</p>")]
        else:
            for video in filtered_files:
                checkbox = widgets.Checkbox(
                    value=False,
                    description=f"Skip: {video['name']} | Size: {video['size_mb']} MB | Duration: {video['duration_min']} minutes",
                    indent=False
                )
                self.skip_checkboxes.append(checkbox)

            skip_selection_box = widgets.VBox(children=self.skip_checkboxes)
            download_batch_button = widgets.Button(
                description=f"🚀 Start Download Selected Files ({self.download_limit_per_batch} files at a time)",
                button_style='success'
            )
            download_all_button = widgets.Button(
                description=f"⬇️ Download All ({self.download_limit_per_batch} files at a time)",
                button_style='warning',
                tooltip="Download all currently displayed files (without skipping)"
            )

            # Remove old handlers to prevent duplicates
            if hasattr(download_batch_button, '_click_handlers'):
                download_batch_button.on_click(self._on_download_batch_click, remove=True)
            if hasattr(download_all_button, '_click_handlers'):
                download_all_button.on_click(self._on_download_all_click, remove=True)

            download_batch_button.on_click(self._on_download_batch_click)
            download_all_button.on_click(self._on_download_all_click)

            self.batch_download_container.children = [
                skip_selection_box,
                widgets.HBox([download_batch_button, download_all_button])
            ]

    def _create_single_download_widget(self, file_info):
        """Creates a button and info for single file download."""
        file_label = widgets.HTML(
            value=f"<span>📁 <b>{file_info['name']}</b> | Size: {file_info['size_mb']} MB | Duration: {file_info['duration_min']} minutes</span>"
        )
        download_button = widgets.Button(description="⬇️ Download", button_style='info')

        def on_button_click(b):
            b.disabled = True
            with self.output_area:
                clear_output(wait=True)
                print(f"Preparing to download: {file_info['name']}...")
                try:
                    google.colab.files.download(file_info['path'])
                    with open(self.download_links_file, "a") as f:
                        f.write(f"{file_info['path']}\n")
                    print(f"✅ Download of {file_info['name']} initiated! Check your browser's download bar. (URL saved)")
                    self._display_download_history()
                except Exception as e:
                    print(f"❌ Error downloading {file_info['name']}: {e}")
                finally:
                    b.disabled = False

        if hasattr(download_button, '_click_handlers'):
            download_button.on_click(on_button_click, remove=True)
        download_button.on_click(on_button_click)
        return widgets.HBox([download_button, file_label])

    def _on_download_batch_click(self, b):
        """Handles the batch download button click (skipping selected)."""
        b.disabled = True
        with self.output_area:
            clear_output(wait=True)
            files_to_download = []
            # Use the original video_files_info to correctly map checkboxes to files
            # This handles cases where filtering might have changed the order/count
            for i, checkbox in enumerate(self.skip_checkboxes):
                # Find the corresponding file in the original list based on description
                # This assumes descriptions are unique enough or we need a better mapping
                # For simplicity, let's re-filter video_files_info based on current filter and then map checkboxes
                filtered_files = [
                    file_info for file_info in self.video_files_info
                    if self.current_filter_text in file_info['name'].lower()
                ]
                if i < len(filtered_files) and not checkbox.value:
                    files_to_download.append(filtered_files[i])

            if not files_to_download:
                print("🚫 No files to download. Please select at least one file or uncheck skip options.")
                b.disabled = False
                return

            self._initiate_batch_download(files_to_download, b)

    def _on_download_all_click(self, b):
        """Handles the 'Download All' button click."""
        b.disabled = True
        with self.output_area:
            clear_output(wait=True)
            # Download all currently filtered files
            files_to_download = [
                file_info for file_info in self.video_files_info
                if self.current_filter_text in file_info['name'].lower()
            ]
            if not files_to_download:
                print("🚫 No files to download.")
                b.disabled = False
                return
            self._initiate_batch_download(files_to_download, b)


    def _initiate_batch_download(self, files_to_download, button_widget):
        """Initiates the batch download process."""
        with self.output_area:
            print(f"📦 Preparing {len(files_to_download)} selected files for download...")

            downloaded_count = 0
            total_files = len(files_to_download)
            try:
                for i, file_info in enumerate(files_to_download):
                    if downloaded_count >= self.download_limit_per_batch:
                        print(f"\n⚠️ Stop! Reached the download limit of {self.download_limit_per_batch} files per batch.")
                        print("Please finish downloading the ready files, then click the '🚀 Start Download Selected Files' button again to download the remaining files.")
                        break

                    print(f"\n({i + 1}/{total_files}) Downloading: {file_info['name']}...")
                    try:
                        google.colab.files.download(file_info['path'])
                        with open(self.download_links_file, "a") as f:
                            f.write(f"{file_info['path']}\n")
                        print(f"✅ Download of {file_info['name']} initiated! (URL saved)")
                        self._display_download_history()
                    except Exception as e:
                        print(f"❌ Error downloading {file_info['name']}: {e}")

                    downloaded_count += 1
                    time.sleep(1) # Add a slightly longer delay between downloads

                if downloaded_count < total_files and downloaded_count < self.download_limit_per_batch:
                    print("\n🎉 File preparation for download completed!")
                elif downloaded_count == total_files:
                    print("\n🎉 All selected files have been prepared for download!")
            finally:
                button_widget.disabled = False # Re-enable button

    def _display_download_history(self):
        """Displays the content of the downloaded links file."""
        with self.download_history_output:
            clear_output(wait=True)
            try:
                with open(self.download_links_file, "r") as f:
                    history = f.read().strip()
                if history:
                    print("Downloaded Files:")
                    print(history)
                else:
                    print("No files downloaded yet.")
            except FileNotFoundError:
                print("No files downloaded yet (history file not found).")
            except Exception as e:
                print(f"❌ Error reading download history: {e}")

    def display(self):
        """Displays the main application UI."""
        display(self.main_layout)

# --- Instantiate and Run the Application ---
print("Initializing Colab Video Downloader...")
app = ColabVideoDownloader(
    download_dir=default_download_dir,
    download_limit_per_batch=download_limit_per_batch,
    video_extensions=video_extensions,
    download_links_file=download_links_file
)
app.display()
print("\n--- Setup Complete ---")
print("👆 You can scroll up to the 'Colab Video Downloader (Epic Version)' section to use it!")


In [ ]:
from google.colab import drive
import os
import shutil
import subprocess
from datetime import timedelta
import pandas as pd
import logging
from tqdm.notebook import tqdm # Use tqdm.notebook for Google Colab
import hashlib # For checksum verification

# --- Configuration ---
# Source directory path where your videos are located
SOURCE_DIR = '/content/DouyinLiveRecorder/downloads/抖音直播'
# Target directory path in Google Drive where backups will be stored
TARGET_DIR = '/content/drive/MyDrive/抖音直播备份' # Can be modified as needed

# Advanced Configuration
ENABLE_CHECKSUM_VERIFICATION = True # Set to True to verify file integrity after copying
LOG_TO_FILE = True # Set to True to save logs to a file in the target directory
LOG_FILE_NAME = 'backup_log.log' # Name of the log file if LOG_TO_FILE is True
# --------------------

# Set up logging
# Remove any existing handlers to prevent duplicate logs if the cell is run multiple times
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to install FFmpeg if not present
def install_ffmpeg():
    """
    Checks if ffprobe is installed and installs FFmpeg if it's not found.
    FFmpeg includes ffprobe, which is necessary for getting video durations.
    """
    try:
        subprocess.run(['ffprobe', '-h'], check=True, capture_output=True)
        logging.info("FFmpeg (ffprobe) is already installed.")
    except (subprocess.CalledProcessError, FileNotFoundError):
        logging.warning("FFmpeg (ffprobe) not found. Attempting to install FFmpeg...")
        try:
            # Update apt-get and install ffmpeg
            subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
            logging.info("FFmpeg installed successfully.")
        except subprocess.CalledProcessError as e:
            logging.error(f"Failed to install FFmpeg: {e.stderr.decode()}. Please install it manually.")
            raise RuntimeError("FFmpeg installation failed.")

# Function to calculate MD5 checksum of a file
def calculate_md5(file_path, chunk_size=8192):
    """
    Calculates the MD5 checksum of a given file.

    Args:
        file_path (str): The path to the file.
        chunk_size (int): The size of chunks to read from the file.

    Returns:
        str: The MD5 checksum in hexadecimal format, or None if an error occurs.
    """
    try:
        hasher = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hasher.update(chunk)
        return hasher.hexdigest()
    except Exception as e:
        logging.error(f"Error calculating MD5 for {file_path}: {e}")
        return None

# Mount Google Drive
logging.info("Mounting Google Drive...")
try:
    drive.mount('/content/drive')
    logging.info("Google Drive mounted successfully.")
except Exception as e:
    logging.error(f"Failed to mount Google Drive: {e}")
    raise

# Ensure FFmpeg is installed
install_ffmpeg()

# Check if source directory exists
if not os.path.exists(SOURCE_DIR):
    logging.error(f"Source directory not found: {SOURCE_DIR}")
    raise FileNotFoundError(f"Source directory not found: {SOURCE_DIR}")
logging.info(f"Source directory found: {SOURCE_DIR}")

# Create target directory (if it doesn't exist)
try:
    os.makedirs(TARGET_DIR, exist_ok=True)
    logging.info(f"Checked and created target directory: {TARGET_DIR}")
except Exception as e:
    logging.error(f"Failed to create target directory: {e}")
    raise

# Configure file logging if enabled
if LOG_TO_FILE:
    log_file_path = os.path.join(TARGET_DIR, LOG_FILE_NAME)
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logging.getLogger().addHandler(file_handler)
    logging.info(f"Logging to file enabled: {log_file_path}")

# Get video file duration (using ffprobe)
def get_video_duration(file_path):
    """
    Retrieves the duration of a video file using ffprobe.

    Args:
        file_path (str): The path to the video file.

    Returns:
        str: The duration in HH:MM:SS format, or "N/A" if an error occurs.
    """
    try:
        cmd = [
            'ffprobe', '-v', 'error', '-show_entries',
            'format=duration', '-of',
            'default=noprint_wrappers=1:nokey=1', file_path
        ]
        duration_str = subprocess.check_output(cmd, stderr=subprocess.PIPE).decode('utf-8').strip()
        duration = float(duration_str)
        return str(timedelta(seconds=duration))
    except (subprocess.CalledProcessError, ValueError) as e:
        logging.warning(f"Could not get duration for file {file_path}: {e}. Returning 'N/A'.")
        return "N/A"
    except FileNotFoundError:
        # This error should ideally be caught by install_ffmpeg, but kept for robustness
        logging.error(" 'ffprobe' not found. Please ensure FFmpeg is installed.")
        return "N/A"

# Scan video files and return as DataFrame
def scan_video_files(directory):
    """
    Scans a directory for video files (.mp4, .ts) and collects their information.

    Args:
        directory (str): The directory to scan.

    Returns:
        pd.DataFrame: A DataFrame containing video file information.
    """
    logging.info(f"Scanning video files in: {directory}")
    video_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.mp4', '.ts')):
                file_path = os.path.join(root, file)
                try:
                    size = os.path.getsize(file_path) / (1024 * 1024) # MB
                    duration = get_video_duration(file_path)
                    video_files.append({
                        "File Name": file,
                        "Path": file_path,
                        "Size (MB)": round(size, 2),
                        "Duration": duration
                    })
                except OSError as e:
                    logging.warning(f"Could not access file {file_path}: {e}. Skipping this file.")
    logging.info(f"Scan complete. Found {len(video_files)} video files.")
    return pd.DataFrame(video_files)

# Scan and display file information
df = scan_video_files(SOURCE_DIR)
if len(df) > 0:
    print("\n📁 Found video files:")
    # Add an Index column for selection
    df_display = df.reset_index().rename(columns={'index': 'Index'})
    print(df_display[['Index', 'File Name', 'Size (MB)', 'Duration']].to_markdown(index=False))
else:
    print("❌ No MP4/TS video files found in the source directory.")
    logging.info("No MP4/TS video files found.")

# Select files to copy
copied_count = 0
skipped_count = 0
failed_count = 0
verified_count = 0
checksum_mismatch_count = 0

if len(df) > 0:
    print("\n🔄 Please select files to copy (enter Index numbers, comma-separated, or 'all' to copy all):")
    choice = input("Your choice: ").strip()

    selected_files_paths = []
    if choice.lower() == 'all':
        selected_files_paths = df['Path'].tolist()
    else:
        try:
            selected_indices = [int(i.strip()) for i in choice.split(',')]
            for idx in selected_indices:
                if 0 <= idx < len(df):
                    selected_files_paths.append(df.iloc[idx]['Path'])
                else:
                    print(f"⚠️ Invalid Index number {idx}. It will be skipped.")
                    logging.warning(f"User entered invalid Index: {idx}")
        except ValueError:
            print("❌ Invalid input. Please enter Index numbers or 'all'.")
            logging.error(f"User entered invalid input: {choice}")
            selected_files_paths = []

    if selected_files_paths:
        print(f"\nCopying {len(selected_files_paths)} files...")
        for src in tqdm(selected_files_paths, desc="Copying"):
            file_name = os.path.basename(src)
            dst = os.path.join(TARGET_DIR, file_name)

            if os.path.exists(dst):
                print(f"⏩ Skipping: '{file_name}' already exists in the target.")
                logging.info(f"Skipping existing file: {file_name}")
                skipped_count += 1
                continue

            try:
                shutil.copy2(src, dst)
                logging.info(f"Successfully copied: {file_name}")
                copied_count += 1

                if ENABLE_CHECKSUM_VERIFICATION:
                    src_md5 = calculate_md5(src)
                    dst_md5 = calculate_md5(dst)

                    if src_md5 and dst_md5 and src_md5 == dst_md5:
                        print(f"✅ Copied and Verified: '{file_name}' (MD5 Match)")
                        logging.info(f"Checksum verified for {file_name}: MD5 Match.")
                        verified_count += 1
                    else:
                        print(f"⚠️ Copied but Checksum Mismatch: '{file_name}' (Source MD5: {src_md5}, Dest MD5: {dst_md5})")
                        logging.warning(f"Checksum mismatch for {file_name}. Source MD5: {src_md5}, Dest MD5: {dst_md5}")
                        checksum_mismatch_count += 1
                else:
                    print(f"✅ Copied: '{file_name}'")

            except Exception as e:
                print(f"❌ Failed to copy: '{file_name}' - {e}")
                logging.error(f"Failed to copy: {file_name} - {e}")
                failed_count += 1
    else:
        print("No files selected for copying.")
        logging.info("No files selected for copying.")

    # Option to delete source files
    if copied_count > 0:
        print("\n❓ Do you want to delete the successfully copied source files? (y/n)")
        delete_choice = input("Your choice: ").strip().lower()
        if delete_choice == 'y':
            deleted_count = 0
            print("\nDeleting source files...")
            # Filter selected_files_paths to only include those that were successfully copied
            files_to_delete = [
                src for src in selected_files_paths
                if os.path.exists(os.path.join(TARGET_DIR, os.path.basename(src)))
            ]

            for src in tqdm(files_to_delete, desc="Deleting"):
                file_name = os.path.basename(src)
                try:
                    os.remove(src)
                    print(f"🗑️ Deleted: '{file_name}'")
                    logging.info(f"Successfully deleted source file: {file_name}")
                    deleted_count += 1
                except Exception as e:
                    print(f"⚠️ Failed to delete: '{file_name}' - {e}")
                    logging.warning(f"Failed to delete source file: {file_name} - {e}")
            print(f"\n🗑️ Deleted {deleted_count} source files.")
            logging.info(f"Deletion summary: Deleted {deleted_count} files.")
        else:
            print("Source files were not deleted.")
            logging.info("User chose not to delete source files.")

# Summary Report
print("\n--- Summary Report ---")
print(f"✅ Successfully copied: {copied_count} files")
if ENABLE_CHECKSUM_VERIFICATION:
    print(f"✔️ Successfully verified (MD5 Match): {verified_count} files")
    print(f"❗ Checksum Mismatch (copied but not verified): {checksum_mismatch_count} files")
print(f"⏩ Skipped (already exists in target): {skipped_count} files")
print(f"❌ Failed to copy: {failed_count} files")
logging.info(f"Operation Summary: Copied={copied_count}, Verified={verified_count}, Mismatch={checksum_mismatch_count}, Skipped={skipped_count}, Failed={failed_count}")

# Unmount Google Drive (optional)
logging.info("Unmounting Google Drive...")
try:
    drive.flush_and_unmount()
    logging.info("Google Drive unmounted successfully.")
except Exception as e:
    logging.error(f"Failed to unmount Google Drive: {e}")

print("\n🎉 Operation complete!")
